## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import sparse

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'NumCompaniesWorked', 'DistanceFromHome', 'Education', 'TotalWorkingYears', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'JobSatisfaction', 'WorkLifeBalance']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Age                        int64
NumCompaniesWorked         int64
DistanceFromHome           int64
Education                  int64
TotalWorkingYears          int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
YearsWithCurrManager       int64
JobSatisfaction            int64
WorkLifeBalance            int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [6]:
# Convert your X data to numeric data types however you see fit
X_train = tf.cast(X_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)


In [7]:
# # Create a StandardScaler
# scaler = StandardScaler()

# # Fit the StandardScaler to the training data
# X_scaler = scaler.fit(X_train)

# # Scale the training and testing data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Normalize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

# Fit the encoder to the training data
enc.fit(y_train[['Department']])

# Create two new variables by applying the encoder to the training and testing data
X_train_encoded = enc.transform(y_train[['Department']])
X_test_encoded = enc.transform(y_test[['Department']])


In [9]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder()

# Fit the encoder to the training data
enc.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder to the training and testing data
encoded_y_train = enc.transform(y_train[['Attrition']])
encoded_y_test = enc.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
n_cols = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(n_cols,))

# Create at least two shared layers
shared_layer1 = layers.Dense(10, activation='relu')(input_layer)
shared_layer2 = layers.Dense(10, activation='relu')(shared_layer1)


In [11]:
# Create a branch for Department with a hidden layer and an output layer
department_hidden = layers.Dense(10, activation='relu')(shared_layer2)
department_output = layers.Dense(3, activation='sigmoid', name='department')(department_hidden)


In [12]:
# Explicitly ensure it is dense and the correct data type
attrition_dense_input = tf.convert_to_tensor(shared_layer2)
attrition_dense_input = tf.cast(attrition_dense_input, dtype=tf.float32)

# Then continue with your model definition
attrition_hidden = layers.Dense(10, activation='relu')(attrition_dense_input)
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition')(attrition_hidden)

In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam', loss={'department': 'categorical_crossentropy', 'attrition': 'categorical_crossentropy'}, metrics=['accuracy'])

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 10)                   110       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 10)                   110       ['dense[0][0]']               
                                                                                                  
 tf.convert_to_tensor (TFOp  (None, 10)                   0         ['dense_1[0][0]']             
 Lambda)                                                                                      

In [25]:
# Convert scipy sparse matrices to TensorFlow SparseTensors
encoded_y_train_sparse = tf.sparse.from_dense(encoded_y_train.todense())
encoded_y_test_sparse = tf.sparse.from_dense(encoded_y_test.todense())

# Convert SparseTensors to dense tensors
encoded_y_train_dense = tf.sparse.to_dense(encoded_y_train_sparse)
encoded_y_test_dense = tf.sparse.to_dense(encoded_y_test_sparse)

# Convert dense tensors to one-hot encoded tensors
encoded_y_train_one_hot = tf.one_hot(encoded_y_train_dense, depth=2)
encoded_y_test_one_hot = tf.one_hot(encoded_y_test_dense, depth=2)

# Evaluate the model with the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled,
    {
        "department": X_test_encoded,
        "attrition": encoded_y_test_one_hot,
    },
    verbose=2,
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

InvalidArgumentError: Value for attr 'TI' of double is not in the list of allowed values: uint8, int8, int32, int64
	; NodeDef: {{node OneHot}}; Op<name=OneHot; signature=indices:TI, depth:int32, on_value:T, off_value:T -> output:T; attr=axis:int,default=-1; attr=T:type; attr=TI:type,default=DT_INT64,allowed=[DT_UINT8, DT_INT8, DT_INT32, DT_INT64]> [Op:OneHot] name: 

In [14]:
# Train the model
model.fit(X_train_scaled, {'department': encoded_y_train, 'attrition': encoded_y_train}, epochs=100, shuffle=True, verbose=2)


Epoch 1/100


TypeError: in user code:

    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/jake/anaconda3/envs

In [ ]:
# Evaluate the model with the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, {'department': encoded_y_test, 'attrition': encoded_y_test}, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

TypeError: in user code:

    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/jake/anaconda3/envs/scikit_dev/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/jake/anaconda3/envs/scikit_de

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {model_accuracy[1]}, Attrition Accuracy: {model_accuracy[2]}")


NameError: name 'model_accuracy' is not defined

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 